In [2]:
"""
N-CMAPSS DS08a - RUL 예측 및 경보 시스템
- Update: Piecewise RUL Max 30으로 조정 (단기 예측 집중)
- Update: Val RMSE > 8.0 발생 시 조기 종료 (Early Stopping) 로직 추가
- Base: Data Augmentation, Baseline Centering, Model Lightening 적용됨
"""

# 구글 코랩 드라이브 마운트
try:
    from google.colab import drive
    drive.mount('/content/drive')
    print("✅ Google Drive mounted!")
except:
    print("⚠️ Not in Colab")

import h5py
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from tqdm.auto import tqdm
import os
import warnings

warnings.filterwarnings('ignore')

# =============================
# 0. 설정 (Configuration)
# =============================
H5_PATH = "/content/drive/MyDrive/LIZaiMing/N-CMAPSS_DS08a-009.h5"

WINDOW_SIZE = 30
STRIDE = 1
BATCH_SIZE = 256
EPOCHS = 20
LEARNING_RATE = 0.001

# [수정] Piecewise RUL 설정 (50 -> 30)
# 이제 모델은 30 사이클 이하(고장 임박) 구간만 정밀하게 예측하고,
# 30 이상은 모두 '안전(30)'으로 간주합니다.
PIECEWISE_RUL_MAX = 30.0

DANGER_THRESHOLD_RATIO = 0.10

# 노이즈 강도 설정
NOISE_FACTOR = 0.05

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}\n")

def set_korean_font():
    if 'google.colab' in str(get_ipython()):
        if not os.path.exists('/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'):
            os.system("sudo apt-get install -y fonts-nanum")
            os.system("sudo fc-cache -fv")
            os.system("rm ~/.cache/matplotlib -rf")
        font_path = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
        plt.rc('font', family=fm.FontProperties(fname=font_path).get_name())
        plt.rcParams['axes.unicode_minus'] = False
    else:
        plt.rcParams['font.family'] = 'Malgun Gothic'

set_korean_font()

# =============================
# 1. 데이터 로드 및 RUL 라벨 생성
# =============================
def load_and_process_data():
    print("[1] 데이터 로딩 및 RUL 계산 중...")
    if not os.path.exists(H5_PATH):
        raise FileNotFoundError(f"파일을 찾을 수 없습니다: {H5_PATH}")

    with h5py.File(H5_PATH, "r") as f:
        W = f["W_dev"][:].astype(np.float32)
        Xs = f["X_s_dev"][:].astype(np.float32)
        X = np.concatenate([W, Xs], axis=1)

        aux = f["A_dev"][:]
        unit_ids = aux[:, 0].astype(int)
        cycles = aux[:, 1].astype(int)

    print(f"  X shape: {X.shape}")

    RUL = np.zeros(len(cycles))
    max_life_dict = {}

    for unit in tqdm(np.unique(unit_ids), desc="Generating RUL Labels"):
        mask = (unit_ids == unit)
        unit_cycles = cycles[mask]
        max_cycle = unit_cycles.max()
        max_life_dict[unit] = max_cycle

        linear_rul = max_cycle - unit_cycles
        RUL[mask] = np.clip(linear_rul, 0, PIECEWISE_RUL_MAX)

    print(f"✅ Piecewise RUL 생성 완료 (Max Limit: {PIECEWISE_RUL_MAX})\n")
    return X, RUL, unit_ids, max_life_dict

# =============================
# 1-1. Baseline Centering
# =============================
def apply_baseline_centering(X, unit_ids, n_init_samples=50):
    print("[Pre-processing] Baseline Centering 적용 중...")
    X_centered = np.zeros_like(X)

    for unit in tqdm(np.unique(unit_ids), desc="Centering"):
        mask = (unit_ids == unit)
        X_unit = X[mask]

        n_samples = min(len(X_unit), n_init_samples)
        baseline = X_unit[:n_samples].mean(axis=0)
        X_centered[mask] = X_unit - baseline

    print("✅ Baseline Centering 완료\n")
    return X_centered

# =============================
# 2. 윈도우 생성
# =============================
def create_windows(X, y, unit_ids, window_size, stride):
    windows_X = []
    windows_y = []
    windows_unit = []

    for unit in np.unique(unit_ids):
        mask = (unit_ids == unit)
        X_unit = X[mask]
        y_unit = y[mask]

        for i in range(0, len(X_unit) - window_size + 1, stride):
            windows_X.append(X_unit[i : i + window_size])
            windows_y.append(y_unit[i + window_size - 1])
            windows_unit.append(unit)

    return np.array(windows_X), np.array(windows_y), np.array(windows_unit)

# =============================
# 3. 데이터셋 클래스 (Noise Injection 추가)
# =============================
class RULDataset(Dataset):
    def __init__(self, X, y, mode='train'):
        self.X = torch.FloatTensor(X)
        self.y = torch.FloatTensor(y).unsqueeze(1)
        self.mode = mode # 'train' or 'test'

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        x_data = self.X[idx]
        y_data = self.y[idx]

        # 학습 모드일 때만 노이즈 추가 (Data Augmentation)
        if self.mode == 'train':
            noise = torch.randn_like(x_data) * NOISE_FACTOR
            x_data = x_data + noise

        return x_data, y_data

# =============================
# 4. LSTM Regressor 모델 (경량화 유지)
# =============================
class LSTMRegressor(nn.Module):
    def __init__(self, input_dim, hidden_dim=32):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=1,
            batch_first=True,
            dropout=0.0,
            bidirectional=False
        )
        self.dropout = nn.Dropout(0.6)
        self.fc = nn.Sequential(
            nn.Linear(hidden_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 1)
        )

    def forward(self, x):
        out, _ = self.lstm(x)
        last_out = out[:, -1, :]
        last_out = self.dropout(last_out)
        pred = self.fc(last_out)
        return pred

# =============================
# 5. 학습 함수 (Early Stopping 추가)
# =============================
def train_model(model, train_loader, val_loader):
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-2)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=2, factor=0.5)

    best_loss = float('inf')

    print(f"\n[모델 학습 시작] Epochs: {EPOCHS} (Noise Factor: {NOISE_FACTOR})")
    for epoch in range(EPOCHS):
        model.train()
        train_loss = 0

        for X_batch, y_batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}", leave=False):
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            optimizer.zero_grad()
            preds = model(X_batch)
            loss = criterion(preds, y_batch)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        train_loss /= len(train_loader)

        model.eval()
        val_loss = 0
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                preds = model(X_batch)
                loss = criterion(preds, y_batch)
                val_loss += loss.item()

        val_loss /= len(val_loader)
        train_rmse = np.sqrt(train_loss)
        val_rmse = np.sqrt(val_loss)

        print(f"Epoch {epoch+1}: Train RMSE={train_rmse:.2f} | Val RMSE={val_rmse:.2f}")

        # [NEW] Early Stopping Check (RMSE > 8)
        if val_rmse > 8.0:
            print(f"\n🚨 [Early Stopping] Val RMSE({val_rmse:.2f})가 임계값(8.0)을 초과했습니다.")
            print("   과적합 방지를 위해 학습을 조기 종료합니다.")
            break

        scheduler.step(val_loss)

        if val_loss < best_loss:
            best_loss = val_loss
            torch.save(model.state_dict(), 'best_rul_model.pt')
            print("  ✓ Best Model Saved")

    print("✅ 학습 종료. (저장된 Best Model로 평가를 진행합니다)")

# =============================
# 6. 평가 및 시각화
# =============================
def evaluate_and_plot(model, X_test, y_test, unit_test, max_life_dict):
    model.eval()

    # Test 시에는 노이즈 없이 평가 (mode='test')
    test_ds = RULDataset(X_test, y_test, mode='test')
    test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False)

    all_preds = []
    with torch.no_grad():
        for X_batch, _ in test_loader:
            X_batch = X_batch.to(device)
            preds = model(X_batch)
            all_preds.append(preds.cpu().numpy())

    all_preds = np.concatenate(all_preds).flatten()

    mse = mean_squared_error(y_test, all_preds)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, all_preds)
    r2 = r2_score(y_test, all_preds)

    print("\n" + "="*50)
    print(" [최종 모델 성능 평가] ")
    print(f" RMSE (평균 제곱근 오차): {rmse:.4f}")
    print(f" MAE  (평균 절대 오차)  : {mae:.4f}")
    print(f" R2 Score (결정 계수)   : {r2:.4f}")
    print("="*50 + "\n")

    unique_test_units = np.unique(unit_test)

    for unit in unique_test_units:
        mask = (unit_test == unit)
        y_true_unit = y_test[mask]
        y_pred_unit = all_preds[mask]

        max_life = max_life_dict[unit]
        danger_rul = max_life * DANGER_THRESHOLD_RATIO

        plt.figure(figsize=(12, 6))

        plt.plot(y_true_unit, label='Actual RUL (정답)', color='black', linestyle='--')
        plt.plot(y_pred_unit, label='Predicted RUL (예측)', color='blue')

        plt.axhline(y=PIECEWISE_RUL_MAX, color='green', linestyle=':', label=f'Max RUL Limit ({int(PIECEWISE_RUL_MAX)})')
        plt.axhline(y=danger_rul, color='red', linestyle='-', linewidth=2, label=f'Danger Threshold ({int(danger_rul)} cycles)')
        plt.fill_between(range(len(y_true_unit)), 0, danger_rul, color='red', alpha=0.1)

        alarm_idx = np.where(y_pred_unit < danger_rul)[0]
        if len(alarm_idx) > 0:
            first_alarm = alarm_idx[0]
            plt.scatter(first_alarm, y_pred_unit[first_alarm], color='red', s=100, zorder=5, marker='*', label='🚨 Alarm Triggered')
            print(f"\n[Unit {unit}] 🚨 경보 발령! 예측 RUL이 {danger_rul:.1f} 미만으로 떨어졌습니다. (Time: {first_alarm})")

        plt.title(f'Piecewise RUL Prediction - Unit {unit} (Max Life: {max_life})')
        plt.xlabel('Time (Window Step)')
        plt.ylabel('Remaining Useful Life (Cycle)')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.show()

# =============================
# 7. 메인 실행
# =============================
def main():
    X, RUL, unit_ids, max_life_dict = load_and_process_data()

    X = apply_baseline_centering(X, unit_ids, n_init_samples=50)

    unique_units = np.unique(unit_ids)
    test_units = unique_units[-2:]
    train_units = unique_units[:-2]

    print(f"Train Units: {train_units}")
    print(f"Test Units: {test_units}")

    train_mask = np.isin(unit_ids, train_units)
    test_mask = np.isin(unit_ids, test_units)

    X_train_raw, RUL_train_raw = X[train_mask], RUL[train_mask]
    X_test_raw, RUL_test_raw = X[test_mask], RUL[test_mask]

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_raw)
    X_test_scaled = scaler.transform(X_test_raw)

    print("\n[3] 슬라이딩 윈도우 생성 중...")
    X_train_win, y_train_win, _ = create_windows(X_train_scaled, RUL_train_raw, unit_ids[train_mask], WINDOW_SIZE, STRIDE)
    X_test_win, y_test_win, u_test_win = create_windows(X_test_scaled, RUL_test_raw, unit_ids[test_mask], WINDOW_SIZE, STRIDE)

    print(f"  Train Windows: {X_train_win.shape}")
    print(f"  Test Windows:  {X_test_win.shape}")

    # Train Loader 생성 시 mode='train'으로 노이즈 주입
    train_ds = RULDataset(X_train_win, y_train_win, mode='train')
    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)

    # Val Loader 생성 시 mode='test'로 노이즈 제거 (정확한 평가를 위해)
    val_ds = RULDataset(X_test_win, y_test_win, mode='test')
    val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

    model = LSTMRegressor(input_dim=X.shape[1]).to(device)
    train_model(model, train_loader, val_loader)

    # 조기 종료 되더라도 저장된 베스트 모델 로드 시도
    if os.path.exists('best_rul_model.pt'):
        model.load_state_dict(torch.load('best_rul_model.pt'))
        print("\n🔄 Best Model 로드 완료")
    else:
        print("\n⚠️ 저장된 Best Model이 없습니다. 현재 모델로 평가합니다.")

    evaluate_and_plot(model, X_test_win, y_test_win, u_test_win, max_life_dict)

if __name__ == "__main__":
    main()

17464789/17464789 [==============================] - 3s 0us/step
